### Different params for transformer encoder. Increase # of heads.

In [8]:
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import math

from scipy.linalg import expm, hadamard, signm
import numpy as np

import os

import seaborn as sns

import logging
import math

import matplotlib.pyplot as plt

In [9]:
import sys
sys.path.insert(0, '/home/rahulpadmanabhan/Development/ws1/masters_thesis_2/LAWT/src/neuralnet')
# sys.path.insert(0, '/home/rahulpadmanabhan/Development/ws1/masters_thesis_2/LAWT/src/')
import datagenerator
from models import MatrixFunctionTransformer
from common import get_logger

In [10]:
torch.set_default_dtype(torch.float64)

In [11]:
save_dir = "/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843"

In [12]:
logger = get_logger()

In [13]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

2024-09-09 00:26:18,510 - root - INFO - Using device: cuda


# Settings

In [24]:
dim = 3
operation = "sign"

In [25]:
test_samples = 2**15  # 16384
k_values = range(5, 19)
train_samples = [32768]
dim = dim

In [26]:
logger.info(f"{train_samples=}, {test_samples=}")

relative_errors = []

2024-09-09 00:30:21,036 - root - INFO - train_samples=[32768], test_samples=32768


In [27]:
train_vals = [32768 - 16384]
models_lst = [*map(lambda x: os.path.join(save_dir, f"dim_{dim}", f'{operation}_model_{str(x)}.pth'), train_vals)]
train_lst = [*map(lambda x: os.path.join(save_dir, f"dim_{dim}", "train", f'train_dataset_{str(x)}.pt'), train_vals)]
test_lst = [*map(lambda x: os.path.join(save_dir, f"dim_{dim}", "test", f'test_dataset_{str(x)}.pt'), train_vals)]

logger.info(f"{models_lst=}\n\n{train_lst=}\n\n{test_lst=}")

2024-09-09 00:30:21,533 - root - INFO - models_lst=['/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843/dim_3/sign_model_16384.pth']

train_lst=['/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843/dim_3/train/train_dataset_16384.pt']

test_lst=['/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843/dim_3/test/test_dataset_16384.pt']


In [38]:
models_lst[0]

'/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843/dim_3/sign_model_16384.pth'

In [33]:
dim=3
d_model = dim*dim
# Ensure d_model is divisible by nhead
nhead = dim
d_model = (dim // nhead) * nhead
# Load the saved model
model = MatrixFunctionTransformer(d_model, dim, 10).to(device)
model.load_state_dict(torch.load(models_lst[0]))
logger.info(model)

2024-09-09 00:32:08,277 - root - INFO - MatrixFunctionTransformer(
  (encoder1): TransformerEncoder(
    (layers): ModuleList(
      (0-9): 10 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_features=3, out_features=12, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=12, out_features=3, bias=True)
        (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (encoder2): TransformerEncoder(
    (layers): ModuleList(
      (0-9): 10 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)


In [34]:
# models_lst[]
dim=3
d_model = dim*dim
# Ensure d_model is divisible by nhead
nhead = dim
d_model = (dim // nhead) * nhead
model = MatrixFunctionTransformer(d_model, dim, 10).to(device)
model.load_state_dict(torch.load(models_lst[0]))
model.eval()

MatrixFunctionTransformer(
  (encoder1): TransformerEncoder(
    (layers): ModuleList(
      (0-9): 10 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_features=3, out_features=12, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=12, out_features=3, bias=True)
        (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (encoder2): TransformerEncoder(
    (layers): ModuleList(
      (0-9): 10 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_f

In [44]:
mu_lst = []
sigma_lst = []
y_main_lst = []
u_shaded_lst = []
l_shaded_lst = []
dim=3
for idx, train_val in enumerate(train_vals):

    # Load the saved model
    d_model = dim*dim
    # Ensure d_model is divisible by nhead
    nhead = dim
    d_model = (dim // nhead) * nhead
    model = MatrixFunctionTransformer(d_model, dim, 10).to(device)
    model.load_state_dict(torch.load('/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843/dim_3/sign_model_16384.pth'))
    model.eval()

    # Loading the test dataset
    test_dataset = torch.load(test_lst[idx])

    logger.info(f"{idx=}\nLoaded model: {models_lst[idx]}")
    logger.info(f"{len(test_dataset)=}")

    actuals = []
    predicted = []

    logger.info(f"Starting to predict values in the test_dataset")
    with torch.no_grad():
        logger.info(f"{dim=}")
        for x, y in test_dataset:
            x = x.view(-1, dim, dim).to(device).to(torch.float64)
            y = y.view(-1, dim, dim).to(device).to(torch.float64)
            # if dim == 1:
                # predicted.append(model(x.view(-1, 1).to(device).to(torch.float64)))
                # actuals.append(y.view(-1,1).to(device).to(torch.float64))
            # else:
                # predicted.append(model(x.view(1, dim*dim).to(device).to(torch.float64)))
                # actuals.append(y.view(1, dim*dim).to(device).to(torch.float64))
                    
            predicted.append(model(x))
            actuals.append(y)
            
        predicted = torch.cat(predicted, 0)
        actuals = torch.cat(actuals, 0)
        
        logger.info(f"\n{predicted.shape=}\n{actuals.shape=}")
        
        np_predicted = predicted.view(-1).cpu().numpy()
        np_actuals = actuals.view(-1).cpu().numpy()

        logger.info(f"{np_predicted.dtype=},{np_actuals.dtype=}")
        
        # mu = np.mean(np.log10(np_predicted))
        y_is = np.abs(np_predicted - np_actuals)/(np.abs(np_actuals + 1e-6) ) # did this in a hurry, double check denominator
        mu = np.mean(np.log10(y_is))
        sigma = (np_predicted.shape[0] -1)**-1 * np.sum((np.log10(y_is) - mu)**2)
        
        y_main = 10**mu 
        
        u_shaded = 10**(mu - sigma)
        l_shaded = 10**(mu + sigma)
        
        logger.info(f"{mu=},{sigma=}")
        logger.info(f"{y_main=},{u_shaded=},{l_shaded=}")
        
        x_raw = np.repeat(train_vals[idx], np_predicted.shape[0])
        y_raw = np_actuals
        
        logger.info(f"{x_raw.shape=}, {y_raw.shape=}")

        mu_lst.append(mu)
        sigma_lst.append(sigma)
        y_main_lst.append(y_main)
        u_shaded_lst.append(u_shaded)
        l_shaded_lst.append(l_shaded)

2024-09-09 00:46:20,307 - root - INFO - idx=0
Loaded model: /mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sign/encoder_20240908223843/dim_3/sign_model_16384.pth
2024-09-09 00:46:20,307 - root - INFO - len(test_dataset)=32768
2024-09-09 00:46:20,311 - root - INFO - Starting to predict values in the test_dataset
2024-09-09 00:46:20,311 - root - INFO - dim=3
2024-09-09 00:49:53,552 - root - INFO - 
predicted.shape=torch.Size([32768, 3, 3])
actuals.shape=torch.Size([32768, 3, 3])
2024-09-09 00:49:53,553 - root - INFO - np_predicted.dtype=dtype('float64'),np_actuals.dtype=dtype('float64')
2024-09-09 00:49:53,559 - root - INFO - mu=0.5598223915786457,sigma=1.8779931422997431
2024-09-09 00:49:53,560 - root - INFO - y_main=3.6292960126993457,u_shaded=0.04806503349329885,l_shaded=274.04099384705
2024-09-09 00:49:53,560 - root - INFO - x_raw.shape=(294912,), y_raw.shape=(294912,)


In [45]:
mu_lst # Relative error

[0.5598223915786457]

### Observations

Increasing Epochs and heads of attention has a better relative error. Went to only ~55% compared to over 100%